In [ ]:
#1.Generate dataset
#2.train thw classifier and save it
#3.detect the face and name it if it is already sored in a dataset

# generate dataset

In [1]:
import cv2

In [3]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces= face_classifier.detectMultiScale(gray, 1.3, 5)
        #scaling factor= 1.3
        #minimum factor= 5
        if len(faces) == 0:
            return None
        for(x, y, w, h) in faces:
            cropped_face= img[y:y+h,x:x+w]
        return cropped_face
        
    cap = cv2.VideoCapture(0)
    id=1
    img_id= 0
    
    while True:
        ret,frame= cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face= cv2.resize(face_cropped(frame),(200,200))
            face= cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = f"Data/user.{id}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            #(50,50) = ORIGINAL PT. FROM WHERE TEXT TO BE WRITTEN
            #FONT SCALE= 1
            #THICKNESS= 2
            cv2.imshow("cropped_face",face)
            if cv2.waitKey(1)==13 or int(img_id)==200:
                break
    cap.release()
    cv2.destroyAllWindows()
    print("collecting samples is completed.....")
generate_dataset()       

collecting samples is completed.....


# train the classifier and save it

In [3]:
import numpy as np
from PIL import Image
import os
import cv2

In [4]:
def train_classifier(data_dir):
    path = [
    os.path.join(data_dir, f)
    for f in os.listdir(data_dir)
    if f.lower().endswith(".jpg")
]

    faces=[]
    ids=[]

    for image in path:
        img= Image.open(image).convert('L')
        imageNp= np.array(img,'uint8')
        id= int(os.path.split(image)[1].split(".")[1])
        faces.append(imageNp)
        ids.append(id)
    ids= np.array(ids)

    #train the classifier and save
    clf= cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
train_classifier("Data")

# detect the face and name it

In [2]:
import numpy as np
from PIL import Image
import os
import cv2
from datetime import datetime
import csv


In [3]:
marked_names = set()
def mark_attendance(name):
    if name not in marked_names:
        now = datetime.now()
        date = now.strftime("%Y-%m-%d")
        time = now.strftime("%H:%M:%S")
        with open("attendance.csv", "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([name, date, time])
        marked_names.add(name)

In [ ]:

def draw_boundary(img, classifier, scaleFactor, minNeighbours, color, clf):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbours)

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x,y), (x+w,y+h), color, 2)

        id, pred = clf.predict(gray_image[y:y+h, x:x+w])
        confidence = int(100 * (1 - pred/300))

        if confidence > 77:
            if id == 1:
                name = "Pranjal"
            elif id == 2:
                name = "Bhumi"
            else:
                name = "Unknown"

            cv2.putText(img, name, (x,y-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1)

            mark_attendance(name)
        else:
            cv2.putText(img, "UNKNOWN", (x,y-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1)

    return img


def recognize(img, clf, faceCascade):
    img = draw_boundary(img, faceCascade, 1.1, 10, (255,255,255), clf)
    return img

    
faceCascade= cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf= cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")


video_capture= cv2.VideoCapture(0)
while True:
    ret,img= video_capture.read()
    img= recognize(img,clf,faceCascade)
    cv2.imshow("face detection", img)

    if cv2.waitKey(1)==13:
        break
video_capture.release()
cv2.destroyAllWindows

<function destroyAllWindows>

: 

In [2]:
!pip install requests

In [1]:
import requests
print("requests installed correctly")

requests installed correctly


In [ ]:
import requests

def get_employee_name(emp_id):
    try:
        # Fetching names from your Employment API
        response = requests.get("https://hrmsapi.leanxpert.in/api/Employment")
        emp_data = response.json().get("data", [])
        for emp in emp_data:
            if emp["employmentId"] == emp_id:
                return emp["empName"]
    except:
        return "User"
    return "Unknown"

In [ ]:
import requests

marked_names = set()

def send_attendance(name):
    if name in marked_names:
        return

    marked_names.add(name)

    url = "http://127.0.0.1:5000/mark_attendance"
    data = {"name": name}

    try:
        requests.post(url, json=data)
    except:
        print("API not reachable")


In [6]:
import requests

def send_attendance(name, user_id):
    url = "http://127.0.0.1:5000/mark_attendance"
    payload = {
        "name": name,
        "id": user_id
    }
    requests.post(url, json=payload)